In [97]:
import pandas as pd
import numpy as np
import re
from sklearn.impute import KNNImputer

In [98]:
df = pd.read_csv('Uncleaned_DS_jobs.csv')
df = pd.DataFrame(df)
df.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


# Cleaning Data

In [99]:
df['Salary Estimate'].unique()

array(['$137K-$171K (Glassdoor est.)', '$75K-$131K (Glassdoor est.)',
       '$79K-$131K (Glassdoor est.)', '$99K-$132K (Glassdoor est.)',
       '$90K-$109K (Glassdoor est.)', '$101K-$165K (Glassdoor est.)',
       '$56K-$97K (Glassdoor est.)', '$79K-$106K (Glassdoor est.)',
       '$71K-$123K (Glassdoor est.)', '$90K-$124K (Glassdoor est.)',
       '$91K-$150K (Glassdoor est.)', '$141K-$225K (Glassdoor est.)',
       '$145K-$225K(Employer est.)', '$79K-$147K (Glassdoor est.)',
       '$122K-$146K (Glassdoor est.)', '$112K-$116K (Glassdoor est.)',
       '$110K-$163K (Glassdoor est.)', '$124K-$198K (Glassdoor est.)',
       '$79K-$133K (Glassdoor est.)', '$69K-$116K (Glassdoor est.)',
       '$31K-$56K (Glassdoor est.)', '$95K-$119K (Glassdoor est.)',
       '$212K-$331K (Glassdoor est.)', '$66K-$112K (Glassdoor est.)',
       '$128K-$201K (Glassdoor est.)', '$138K-$158K (Glassdoor est.)',
       '$80K-$132K (Glassdoor est.)', '$87K-$141K (Glassdoor est.)',
       '$92K-$155K (Glassdo

In [100]:
df['Salary Estimate'] = df['Salary Estimate'].apply(lambda x: x.split('(')[0])
df['Salary Estimate'] = df['Salary Estimate'].apply(lambda x: x.replace('K', '').replace('$', ''))
df['min_salary($K)'] = df['Salary Estimate'].apply(lambda x: x.split('-')[0]).astype(int)
df['max_salary($K)'] = df['Salary Estimate'].apply(lambda x: x.split('-')[1]).astype(int)
df['avg_salary($K)'] = (df['min_salary($K)'] + df['max_salary($K)']) / 2
df.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,min_salary($K),max_salary($K),avg_salary($K)
0,0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna",137,171,154.0
1,1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1,137,171,154.0
2,2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1,137,171,154.0
3,3,Data Scientist,137-171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech...",137,171,154.0
4,4,Data Scientist,137-171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",137,171,154.0


In [101]:
labels = ['low', 'medium', 'high', 'very high', 'extreme']
df['salary_category'] = pd.cut(df['avg_salary($K)'], bins=5, labels=labels)
df.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,min_salary($K),max_salary($K),avg_salary($K),salary_category
0,0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna",137,171,154.0,high
1,1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1,137,171,154.0,high
2,2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1,137,171,154.0,high
3,3,Data Scientist,137-171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech...",137,171,154.0,high
4,4,Data Scientist,137-171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",137,171,154.0,high


In [102]:
df['Company Name'] = df['Company Name'].apply(lambda x: x.split('\n')[0])
df.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,min_salary($K),max_salary($K),avg_salary($K),salary_category
0,0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna",137,171,154.0,high
1,1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1,137,171,154.0,high
2,2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1,137,171,154.0,high
3,3,Data Scientist,137-171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech...",137,171,154.0,high
4,4,Data Scientist,137-171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",137,171,154.0,high


In [103]:
df['Job_State'] = df['Location'].apply(lambda x: x.split(',')[-1])

df['same_state'] = np.where(df.Location == df.Headquarters, 1, 0)

df.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,...,Industry,Sector,Revenue,Competitors,min_salary($K),max_salary($K),avg_salary($K),salary_category,Job_State,same_state
0,0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,"New York, NY","New York, NY",1001 to 5000 employees,1993,...,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna",137,171,154.0,high,NY,1
1,1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,...,Research & Development,Business Services,$1 to $2 billion (USD),-1,137,171,154.0,high,VA,0
2,2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,...,Consulting,Business Services,$100 to $500 million (USD),-1,137,171,154.0,high,MA,1
3,3,Data Scientist,137-171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,...,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech...",137,171,154.0,high,MA,0
4,4,Data Scientist,137-171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51 to 200 employees,1998,...,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",137,171,154.0,high,NY,1


In [104]:
df['company_age'] = df.Founded.apply(lambda x: x if x < 1 else 2024 - x)

df['company_age'].sample(10)

512    -1
19      7
203    16
547    27
555    -1
358    -1
637    -1
251    -1
169    21
118     5
Name: company_age, dtype: int64

In [105]:
df['Type of ownership'] = df['Type of ownership'].str.replace('Company - ', '')
df['Type of ownership'].sample(5)

200    Private
137    Private
109     Public
436    Private
470     Public
Name: Type of ownership, dtype: object

In [106]:
df['Size'] = df['Size'].str.replace('employees', '')
df['Size'].sample(10)

309    5001 to 10000 
201        51 to 200 
404    5001 to 10000 
455           10000+ 
350       201 to 500 
175      501 to 1000 
533      501 to 1000 
600       201 to 500 
29         51 to 200 
228      501 to 1000 
Name: Size, dtype: object

In [107]:
def extract_skills(description):
    # List of skills to look for in the job description
    skills = ['Python', 'R', 'SQL', 'Java', 'C++', 'Scala', 'Julia','aws', 'azure', 'excel', 'tableau', 'power bi', 'big data', 'hadoop', 'spark', 'machine learning', 'deep learning', 'nlp', 'data mining', 'statistics', 'mathematics', 'econometrics', 'business intelligence', 'data visualization', 'data analysis', 'data analytics', 'data science', 'data engineering', 'data modeling', 'data wrangling', 'data transformation']
    # Return a list of skills found in the description, case insensitive
    found_skill = [skill.lower() for skill in skills if skill.lower() in description.lower()]
    return ', '.join(found_skill)
df['skills'] = df['Job Description'].apply(extract_skills)
df['skills'].sample(10)

249               python, r, java, c++, machine learning
671    python, r, sql, excel, tableau, statistics, ma...
593    python, r, sql, azure, excel, power bi, machin...
482        r, sql, statistics, mathematics, econometrics
566    python, r, sql, java, c++, scala, aws, big dat...
395    python, r, sql, java, business intelligence, d...
60                     python, r, sql, excel, statistics
310    python, r, sql, java, scala, big data, hadoop,...
450    python, r, sql, java, c++, scala, excel, hadoo...
262    python, r, sql, big data, hadoop, spark, machi...
Name: skills, dtype: object

In [108]:
def extract_experience(description):
    experience = re.search(r'(\d+)\+?\s*(?:year|yr)s?', description, re.IGNORECASE)
    return int(experience.group(1)) if experience else np.nan
df['work_experience'] = df['Job Description'].apply(extract_experience)
df.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,...,Competitors,min_salary($K),max_salary($K),avg_salary($K),salary_category,Job_State,same_state,company_age,skills,work_experience
0,0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,"New York, NY","New York, NY",1001 to 5000,1993,...,"EmblemHealth, UnitedHealth Group, Aetna",137,171,154.0,high,NY,1,31,"r, aws, azure, machine learning, statistics, d...",NaN
1,1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001 to 10000,1968,...,-1,137,171,154.0,high,VA,0,56,"r, sql, big data, hadoop, machine learning, st...",15.0
2,2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,"Boston, MA","Boston, MA",1001 to 5000,1981,...,-1,137,171,154.0,high,MA,1,43,"python, r, c++, aws, excel, machine learning, ...",NaN
3,3,Data Scientist,137-171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000,2000,...,"MKS Instruments, Pfeiffer Vacuum, Agilent Tech...",137,171,154.0,high,MA,0,24,"python, r, sql, aws, excel, machine learning, ...",NaN
4,4,Data Scientist,137-171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51 to 200,1998,...,"Commerce Signals, Cardlytics, Yodlee",137,171,154.0,high,NY,1,26,"python, r, sql, excel, machine learning, stati...",NaN


In [109]:
def title_simplifier(title):
    if 'data scientist' in title.lower():
        return 'data scientist'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'analyst' in title.lower():
        return 'analyst'
    elif 'machine learning' in title.lower():
        return 'mle'
    elif 'manager' in title.lower():
        return 'manager'
    elif 'director' in title.lower():
        return 'director'
    else:
        return 'na'

df['job_simp'] = df['Job Title'].apply(title_simplifier)
df['job_simp'].sample(10)

177    data scientist
127    data scientist
660    data scientist
481    data scientist
399                na
8      data scientist
532    data scientist
451    data scientist
242     data engineer
528    data scientist
Name: job_simp, dtype: object

In [110]:
def seniority(title):
    if 'senior' in title.lower() or 'sr' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
        return 'senior'
    elif 'junior' in title.lower() or 'jr' in title.lower():
        return 'junior'
    else:
        return 'na'
df['seniority'] = df['Job Title'].apply(seniority)
df['seniority'].sample(10)

535        na
255        na
338        na
324        na
354        na
525        na
259        na
496        na
353        na
512    senior
Name: seniority, dtype: object

In [111]:
df['job_simp'].value_counts()

job_simp
data scientist    455
na                 69
analyst            55
data engineer      47
mle                36
manager             7
director            3
Name: count, dtype: int64

In [112]:
df['seniority'].value_counts()

seniority
na        576
senior     94
junior      2
Name: count, dtype: int64

In [113]:
df['Job_State'] = df['Job_State'].apply(lambda x: x.strip())
df['Job_State'] = df['Job_State'].apply(lambda x: x.replace('United States', 'US'))
df['Job_State'] = df['Job_State'].apply(lambda x: x.replace('California', 'CA'))
df['Job_State'] = df['Job_State'].apply(lambda x: x.replace('New Jersey', 'NJ'))
df['Job_State'] = df['Job_State'].apply(lambda x: x.replace('Texas', 'TX'))
df['Job_State'] = df['Job_State'].apply(lambda x: x.replace('Utah', 'UT'))

In [114]:
df['Job_State'].unique()

array(['NY', 'VA', 'MA', 'CA', 'IL', 'MO', 'WA', 'DC', 'Remote', 'TN',
       'TX', 'PA', 'AZ', 'WI', 'GA', 'FL', 'US', 'NE', 'KS', 'NH', 'NJ',
       'LA', 'OH', 'IN', 'MD', 'CO', 'UT', 'OR', 'MI', 'SC', 'MS', 'AL',
       'RI', 'IA', 'MN', 'OK', 'CT', 'NC', 'DE', 'WV'], dtype=object)

In [115]:
df.drop('Founded', axis=1, inplace=True)

In [116]:
def clean_revenue(revenue):

    if pd.isna(revenue) or revenue == '-1':
        return np.nan  
    
    numbers = re.findall(r'\d+', revenue)
    
    if 'million' in revenue.lower():
        bounds = [float(num) * 1e6 for num in numbers]
    elif 'billion' in revenue.lower():
        bounds = [float(num) * 1e9 for num in numbers]
    else:
        return np.nan 
    
    if len(bounds) == 2:
        return np.mean(bounds)  
    elif len(bounds) == 1:
        return bounds[0] 
    else:
        return np.nan
df['Revenue'] = df['Revenue'].apply(clean_revenue)
df['Revenue'].sample(10)

1      1.500000e+09
543    3.750000e+07
35     3.500000e+09
364             NaN
503    3.000000e+06
610    3.000000e+06
261             NaN
183    2.505000e+08
321    1.000000e+10
262             NaN
Name: Revenue, dtype: float64

In [117]:
df['Competitors'] = df['Competitors'].fillna('')
df['Competitors'].astype(str)

df['No_of_Competitors'] = df['Competitors'].apply(lambda x: len(x.split(',')) if x else 0)

df.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Type of ownership,...,avg_salary($K),salary_category,Job_State,same_state,company_age,skills,work_experience,job_simp,seniority,No_of_Competitors
0,0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,"New York, NY","New York, NY",1001 to 5000,Nonprofit Organization,...,154.0,high,NY,1,31,"r, aws, azure, machine learning, statistics, d...",NaN,data scientist,senior,3
1,1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001 to 10000,Public,...,154.0,high,VA,0,56,"r, sql, big data, hadoop, machine learning, st...",15.0,data scientist,na,1
2,2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,"Boston, MA","Boston, MA",1001 to 5000,Private Practice / Firm,...,154.0,high,MA,1,43,"python, r, c++, aws, excel, machine learning, ...",NaN,data scientist,na,1
3,3,Data Scientist,137-171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000,Public,...,154.0,high,MA,0,24,"python, r, sql, aws, excel, machine learning, ...",NaN,data scientist,na,3
4,4,Data Scientist,137-171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51 to 200,Private,...,154.0,high,NY,1,26,"python, r, sql, excel, machine learning, stati...",NaN,data scientist,na,3


In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   index              672 non-null    int64   
 1   Job Title          672 non-null    object  
 2   Salary Estimate    672 non-null    object  
 3   Job Description    672 non-null    object  
 4   Rating             672 non-null    float64 
 5   Company Name       672 non-null    object  
 6   Location           672 non-null    object  
 7   Headquarters       672 non-null    object  
 8   Size               672 non-null    object  
 9   Type of ownership  672 non-null    object  
 10  Industry           672 non-null    object  
 11  Sector             672 non-null    object  
 12  Revenue            432 non-null    float64 
 13  Competitors        672 non-null    object  
 14  min_salary($K)     672 non-null    int64   
 15  max_salary($K)     672 non-null    int64   
 16  avg_sala

# Handling Missing Values

In [119]:
df.replace('Unknown', np.nan, inplace=True)
df.replace('-1', np.nan, inplace=True)
df.replace(-1, np.nan, inplace=True)
df.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Type of ownership,...,avg_salary($K),salary_category,Job_State,same_state,company_age,skills,work_experience,job_simp,seniority,No_of_Competitors
0,0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,"New York, NY","New York, NY",1001 to 5000,Nonprofit Organization,...,154.0,high,NY,1,31.0,"r, aws, azure, machine learning, statistics, d...",NaN,data scientist,senior,3
1,1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001 to 10000,Public,...,154.0,high,VA,0,56.0,"r, sql, big data, hadoop, machine learning, st...",15.0,data scientist,na,1
2,2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,"Boston, MA","Boston, MA",1001 to 5000,Private Practice / Firm,...,154.0,high,MA,1,43.0,"python, r, c++, aws, excel, machine learning, ...",NaN,data scientist,na,1
3,3,Data Scientist,137-171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000,Public,...,154.0,high,MA,0,24.0,"python, r, sql, aws, excel, machine learning, ...",NaN,data scientist,na,3
4,4,Data Scientist,137-171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51 to 200,Private,...,154.0,high,NY,1,26.0,"python, r, sql, excel, machine learning, stati...",NaN,data scientist,na,3


In [120]:
df.isnull().sum()

index                  0
Job Title              0
Salary Estimate        0
Job Description        0
Rating                50
Company Name           0
Location               0
Headquarters          31
Size                  44
Type of ownership     31
Industry              71
Sector                71
Revenue              240
Competitors          501
min_salary($K)         0
max_salary($K)         0
avg_salary($K)         0
salary_category        0
Job_State              0
same_state             0
company_age          118
skills                 0
work_experience      228
job_simp               0
seniority              0
No_of_Competitors      0
dtype: int64

In [121]:
df['company_age'] = df['company_age'].fillna(df['company_age'].median())

df['Rating'] = df['Rating'].fillna(df['Rating'].median())

In [122]:
df['Size'].value_counts()

Size
51 to 200         135
1001 to 5000      104
1 to 50            86
201 to 500         85
10000+             80
501 to 1000        77
5001 to 10000      61
Name: count, dtype: int64

In [123]:
df.fillna({'Size': np.nan}, inplace=True)

In [124]:
knn_imputer = KNNImputer(n_neighbors=5)
df[['work_experience', 'Revenue']] = knn_imputer.fit_transform(df[['work_experience', 'Revenue']])
df['work_experience'] = df['work_experience'].astype(int)

In [125]:
condition_nulls = df.isnull().sum(axis=1) >= 6

# Filter rows containing 0 or -1 in any column
condition_values = df.isin([0, -1]).any(axis=1)

# Combine both conditions
filtered_rows = df[condition_nulls & condition_values]

# Display the filtered DataFrame
df = df.drop(filtered_rows.index)
df.head()


,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Type of ownership,...,avg_salary($K),salary_category,Job_State,same_state,company_age,skills,work_experience,job_simp,seniority,No_of_Competitors
0,0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,"New York, NY","New York, NY",1001 to 5000,Nonprofit Organization,...,154.0,high,NY,1,31.0,"r, aws, azure, machine learning, statistics, d...",6,data scientist,senior,3
1,1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001 to 10000,Public,...,154.0,high,VA,0,56.0,"r, sql, big data, hadoop, machine learning, st...",15,data scientist,na,1
2,2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,"Boston, MA","Boston, MA",1001 to 5000,Private Practice / Firm,...,154.0,high,MA,1,43.0,"python, r, c++, aws, excel, machine learning, ...",2,data scientist,na,1
3,3,Data Scientist,137-171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000,Public,...,154.0,high,MA,0,24.0,"python, r, sql, aws, excel, machine learning, ...",2,data scientist,na,3
4,4,Data Scientist,137-171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51 to 200,Private,...,154.0,high,NY,1,26.0,"python, r, sql, excel, machine learning, stati...",6,data scientist,na,3


In [126]:
df.isnull().sum()

index                  0
Job Title              0
Salary Estimate        0
Job Description        0
Rating                 0
Company Name           0
Location               0
Headquarters           3
Size                  16
Type of ownership      3
Industry              43
Sector                43
Revenue                0
Competitors          473
min_salary($K)         0
max_salary($K)         0
avg_salary($K)         0
salary_category        0
Job_State              0
same_state             0
company_age            0
skills                 0
work_experience        0
job_simp               0
seniority              0
No_of_Competitors      0
dtype: int64

In [128]:
df.to_csv('Cleaned_jobs.csv', index=False)